In [ ]:
!pip install gensim transformers torch scikit-learn


In [ ]:
import pandas as pd
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Load the dataset into a DataFrame
df = pd.read_csv(next(iter(uploaded)))  # Assumes the first uploaded file is your dataset

# Map 'RequirementType' to 'labels' (Functional: 1, Non-Functional: 0)
label_mapping = {'F': 1, 'NF': 0}
df['labels'] = df['RequirementType'].map(label_mapping)

# Check if the 'labels' column was created correctly
print(df[['RequirementType', 'labels']].head())

Saving final_corrected_fine_labeled_reviews.csv to final_corrected_fine_labeled_reviews (1).csv
  RequirementType  labels
0               F       1
1              NF       0
2               F       1
3              NF       0
4              NF       0


In [ ]:
from huggingface_hub import hf_hub_download
from gensim.models import Doc2Vec

# Step 5: Download the model file from Hugging Face
model_path = hf_hub_download(repo_id="RafidMehda/doc2vec_model", filename="doc2vec_model")

# Step 6: Load the fine-tuned Doc2Vec model
doc2vec_model = Doc2Vec.load(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Extract Doc2Vec embeddings for each document in the dataset
doc2vec_embeddings = [doc2vec_model.dv[str(i)] for i in range(len(df))]


In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Step 8: Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def get_distilbert_embeddings(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Get DistilBERT embeddings
    with torch.no_grad():
        outputs = distilbert_model(**inputs)
        last_hidden_state = outputs.last_hidden_state
        pooled_embedding = torch.mean(last_hidden_state, dim=1)  # Average pooling of token embeddings
    return pooled_embedding.squeeze().numpy()

# Step 9: Generate DistilBERT embeddings for the dataset
distilbert_embeddings = [get_distilbert_embeddings(doc) for doc in df['content']]


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, classification_report

# Load the fine-tuned tokenizer and model from Hugging Face
tokenizer = DistilBertTokenizer.from_pretrained('RafidMehda/app_review_model')
model = DistilBertForSequenceClassification.from_pretrained('RafidMehda/app_review_model')

# Ensure the model is on GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Ensure the model is in evaluation mode
model.eval()

# Assuming 'content' is the column that holds the text in your DataFrame
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['labels'], test_size=0.2, random_state=42)

# Check if X_train and X_test are lists of strings
print(X_train[:5])  # Should print some raw text
print(X_test[:5])

# Tokenize the text data from X_train and X_test with reduced max_length
train_inputs = tokenizer(list(X_train), return_tensors="pt", padding=True, truncation=True, max_length=256)
test_inputs = tokenizer(list(X_test), return_tensors="pt", padding=True, truncation=True, max_length=256)

# Move the tokenized inputs to the correct device (GPU/CPU)
test_inputs = {key: val.to(device) for key, val in test_inputs.items()}

# Define a DataLoader for batching to avoid memory issues
batch_size = 8  # Reduced batch size to avoid memory issues
test_dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

predicted_classes = []

# Model evaluation in batches
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        predicted_classes.append(preds)

# Concatenate all batch predictions into a single tensor
predicted_classes = torch.cat(predicted_classes)

# Convert the predicted classes to CPU if needed (for compatibility with sklearn)
preds = predicted_classes.cpu().numpy()

# Evaluate the model's performance
accuracy = accuracy_score(y_test, preds)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Print detailed classification report
print("Classification Report:\n")
print(classification_report(y_test, preds, target_names=['Non-Functional', 'Functional']))


10368    I kinda love this app but the only problem tha...
6515     Have been using it for a week only and will mo...
409                                            Didn't like
4338     limited amount of motivation amd support , i'd...
9485     I have remote learners and this app is so help...
Name: content, dtype: object
1766    It's very easy to add tasks with its incredibl...
9813    Absolutely useless, worked once then never aga...
4921    I bought premium and now I can't login to the ...
4959                                   Yo, fix the widget
9798    Useless app... Niether Reminder nor notificati...
Name: content, dtype: object
Model Accuracy: 99.56%
Classification Report:

                precision    recall  f1-score   support

Non-Functional       1.00      1.00      1.00      1402
    Functional       0.99      1.00      0.99      1097

      accuracy                           1.00      2499
     macro avg       1.00      1.00      1.00      2499
  weighted avg       1.00   

In [ ]:
# Save the fine-tuned model and tokenizer locally
model.save_pretrained('./merge_doc2vec_distilbert_model')
tokenizer.save_pretrained('./merge_doc2vec_distilbert_model')


('./merge_doc2vec_distilbert_model/tokenizer_config.json',
 './merge_doc2vec_distilbert_model/special_tokens_map.json',
 './merge_doc2vec_distilbert_model/vocab.txt',
 './merge_doc2vec_distilbert_model/added_tokens.json')

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

# Login to your Hugging Face account
notebook_login()


In [ ]:
from huggingface_hub import HfApi

# Upload the model to Hugging Face
api = HfApi()
api.upload_folder(
    folder_path='./merge_doc2vec_distilbert_model',  # Path to your fine-tuned model folder
    repo_id='RafidMehda/merge_doc2vec_distilbert_model',  # Your Hugging Face repository
    repo_type='model'
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/RafidMehda/merge_doc2vec_distilbert_model/commit/8573c19342b5007989860e0a85a40020c5a5fdf1', commit_message='Upload folder using huggingface_hub', commit_description='', oid='8573c19342b5007989860e0a85a40020c5a5fdf1', pr_url=None, pr_revision=None, pr_num=None)